# 9. 指令微调（监督微调）



## 本章重点内容

- **指令微调**：利用指令–响应对在预训练模型上做有监督微调，实现对用户指令的精准执行。  
- **损失聚焦**：只计算“回答”部分的损失，强化模型对目标输出的学习。  
- **数据格式**：使用 JSONL 文件，每行一条 `instruction` / `input` / `output` 对。  
- **训练流程**：配置 `TrainingArguments` 并调用 `Trainer`，监控日志或 TensorBoard 上的 loss 曲线。  


## 9.1 指令微调概念简介
欢迎来到本书第三部分“智能体微调篇”的第一章——指令微调。在前面几章中，你已经了解了大模型智能体的闭环思路、生成式大语言模型（LLM）的原理和提示工程的技巧；而要让这些强大的模型真正“听得懂、能执行”业务场景，就必须在预训练能力之上，加上一层专门的“应答训练”。这正是指令微调（Instruction Fine-Tuning）要解决的核心问题。  
首先，我们再简单回顾一下预训练：预训练阶段，模型在海量的无标注文本上自监督学习，通过预测下一个词、填空或重构文本的方式，学会了丰富的语言结构、世界知识和上下文关联。但它的输出更多是“你想知道什么，我就尽量拼凑出相关文字”，缺少对指令意图的专门校准。  
**指令微调**是在预训练基础上，在带有指令提示和相应输出的标记数据集上微调大模型的技术，通过提供一组概述所需操作的指南或指令，使预训练模型适应执行特定任务。它主要解决三个问题：  
- **可控性**（模型能准确理解指令类型）：让模型准确识别“写摘要”“写代码”等不同指令，并按要求输出格式化内容；  
- **一致性**（在不同上下文中保持稳定）：无论上下文如何变化，同一条指令都能得到稳定可靠的回答；  
- **符合人类偏好**（条理清晰、礼貌、不冗余）：通过高质量示例，教会模型在回答时保持条理、礼貌且不啰嗦。

> 预训练就像让学生读遍图书馆的所有书籍，见识广博；而指令微调则好比给这位学生一批历年真题和详细解析，让他在考试中快速抓住考点、条理清晰地作答。  

在接下来的小节里，我们将从原理简介，数据准备、训练流程，到效果评估，一步步带你在有限算力下完成一次完整的指令微调实践。

## 9.2 指令微调原理
指令微调是如何发挥作用的呢？通过在预训练模型上使用专门的“指令-回答”对数据集进行监督学习，这样模型在接受到自然语言指令时能够更加准确且可控地生成预期输出。
![figure1](./png_chap9/pipeline.jpg "指令微调流程")
<p align="center"> 图 1. 指令微调流程</p>

与预训练不同的关键点就藏在优化目标上，指令微调是将“指令+上下文”视作条件输入，仅对“回答”部分计算损失，从而将模型参数从通用语言建模分布迁移到对齐用户意图的分布，同时显著降低对提示中少量示例的依赖并提升零样本泛化能力。

在指令微调阶段，我们的优化目标不再是对所有 token 进行下一个词预测，而是**只对人类标注的“回答”部分**计算交叉熵损失，最大化其对数似然。  

- **预训练阶段**（Next-Token Prediction）  
  $$
    \mathcal{L}_{\mathrm{pre}} \;=\; -\sum_{t=1}^{T} \log P\bigl(x_t \mid x_{1:t-1}\bigr)
  $$

- **指令微调阶段**（Supervised Instruction Fine-Tuning）  
  $$
    \mathcal{L}_{\mathrm{ft}} \;=\; -\sum_{t \in \text{回答}} \log P\bigl(y_t \mid x_{1:t-1}\bigr)
  $$  
  - 这里 \(x\) 包含“指令＋输入＋回答”三部分，  
  - \(y_t\) 仅为“回答”部分的 token，  

## 9.3 指令微调数据集准备

在动手微调前，首先要准备一套高质量的“指令–回答”训练集。本节将从数据来源、数据格式，以及数据质量与多样性三方面，全面介绍如何构建可供指令微调使用的数据集。

### 9.2.1 数据来源
指令微调数据集主要分为三大类：人工构造、蒸馏合成和自我增强合成。

- **人工构造** 人工构造数据完全由人来撰写或从公开资源中手工收集并审核，从头到尾不涉及任何模型生成或机器学习技术，因此数据质量易控但往往规模较小，典型例子有 Natural Instructions、P3 和 Flan-2021 等。
- **蒸馏合成数据** 蒸馏合成数据则借助像 GPT-3/GPT-4 这样的大模型作为“教师”，自动生成大量指令–响应对。与纯人工制作相比，这种方式速度快、成本低，而且能够产出比人力更多样化的示例，常见数据集有 Alpaca、Unnatural Instructions 和 OIG 等。
- **自我增强合成数据**自我增强合成数据的思路是让尚未微调的基础模型（如原始的 GPT-3）在自己生成的输出上不断进行筛选和迭代“自举”——即模型先生成回答，再对结果打分或完善，多轮下来自动构建起高质量的指令–响应对。
![figure2](./png_chap9/self-instruct.jpg "自我增强合成数据流程图")
<p align="center"> 图 2. 自我增强合成数据流程图</p>


下面表格展示了大多数常用数据集的列表：

| 类型                               | 数据集名字                                                                                                                     | 实例数量     | 语言数量    | 构造方式                                   | 是否开源 |
|-----------------------------------|----------------------------------------------------------------------------------------------------------------------------------|-------------:|------------:|--------------------------------------------|---------:|
| **人工构造**                        | [UnifiedQA (Khashabi et al., 2020)](https://github.com/allenai/unifiedqa)                                                         | 750 K        | 英文        | 人工构造                                   | 是        |
|                                    | [UnifiedSKG (Xie et al., 2022)](https://github.com/hkunlp/unifiedskg)                                                              | 0.8 M        | 英文        | 人工构造                                   | 是        |
|                                    | [Natural Instructions (Honovich et al., 2022)](https://github.com/allenai/natural-instructions-v1)                                  | 193 K        | 英文        | 人工构造                                   | 是        |
|                                    | [Super-Natural Instructions (Wang et al., 2022f)](https://github.com/allenai/natural-instructions)                                  | 5 M          | 55 种语言   | 人工构造                                   | 是        |
|                                    | [P3 (Sanh et al., 2021)](https://huggingface.co/datasets/bigscience/P3)                                                           | 12 M         | 英文        | 人工构造                                   | 是        |
|                                    | [xP3 (Muennighoff et al., 2022)](https://github.com/bigscience-workshop/xmtf)                                                      | 81 M         | 46 种语言   | 人工构造                                   | 是        |
|                                    | [Flan 2021 (Longpre et al., 2023)](https://github.com/google-research/FLAN)                                                       | 4.4 M        | 英文        | 人工构造                                   | 是        |
|                                    | [COIG (Zhang et al., 2023a)](https://github.com/BAAI-Zlab/COIG)                                                                   | –            | –           | –                                          | 是        |
|                                    | [InstructGPT (Ouyang et al., 2022)](https://openai.com/research/instructgpt)                                                       | 13 K         | 多语言      | 人工构造                                   | 否        |
|                                    | [Dolly (Conover et al., 2023a)](https://huggingface.co/datasets/databricks/databricks-dolly-15k)                                   | 15 K         | 英文        | 人工构造                                   | 是        |
|                                    | [LIMA (Zhou et al., 2023a)](https://huggingface.co/datasets/GAIR/lima)                                                             | 1 K          | 英文        | 人工构造                                   | 是        |
|                                    | [ChatGPT (OpenAI, 2022)](https://openai.com/blog/chatgpt)                                                                          | –            | 多语言      | 人工构造                                   | 否        |
|                                    | [OpenAssistant (Köpf et al., 2023)](https://huggingface.co/datasets/LAION-AI/OpenAssistant)                                         | 161 443      | 多语言      | 人工构造                                   | 是        |
| **合成数据（蒸馏）**                 | [OIG (LAION.ai, 2023)](https://github.com/LAION-AI/Open-Instruction-Generalist)                                                    | 43 M         | 英文        | ChatGPT（无技术报告）                      | 是        |
|                                    | [Unnatural Instructions (Honovich et al., 2022)](https://github.com/orhonovich/unnatural-instructions)                              | 240 K        | 英文        | InstructGPT 生成                           | 是        |
|                                    | [InstructWild (Xue et al., 2023)](https://github.com/XueFuzhao/InstructionWild)                                                    | 104 K        | –           | ChatGPT 生成                              | 是        |
|                                    | [Evol-Instruct / WizardLM (Xu et al., 2023a)](https://github.com/nlp/cuan-evol-instruct)                                            | 52 K         | 英文        | ChatGPT 生成                              | 是        |
|                                    | [Alpaca (Taori et al., 2023a)](https://github.com/tatsu-lab/stanford_alpaca)                                                       | 52 K         | 英文        | InstructGPT 生成                           | 是        |
|                                    | [LogiCoT (Liu et al., 2023a)](https://github.com/csifun/LogiCoT)                                                                   | –            | 英文        | GPT-4 生成                                | 是        |
|                                    | [GPT-4-LLM (Peng et al., 2023)](https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM)                                        | 52 K         | 英中双语    | GPT-4 生成                                | 是        |
|                                    | [Vicuna (Chiang et al., 2023)](https://lmsys.org/blog/2023-03-30-vicuna)                                                            | 70 K         | 英文        | 实际用户-ChatGPT 对话                      | 否        |
|                                    | [Baize v1 (Conover et al., 2023b)](https://github.com/project-baize/baize-chatbot)                                                  | 111 500      | 英文        | ChatGPT 生成                              | 是        |
|                                    | [UltraChat (Ding et al., 2023a)](https://github.com/thunlp/UltraChat#data)                                                          | 675 K        | 英中双语    | GPT-3/4 生成                              | 是        |
|                                    | [Guanaco (JosephusCheung, 2021)](https://huggingface.co/datasets/JosephusCheung/GuanacoDataset)                                      | 534 530      | 多语言      | GPT（未知版本）生成                       | 是        |
|                                    | [Orca (Mukherjee et al., 2023)](https://huggingface.co/datasets/Open-Orca/OpenOrca)                                                 | 1.5 M        | 英文        | GPT-3.5/4 生成                            | 是        |
|                                    | [ShareGPT](https://huggingface.co/datasets/RyokoAI/ShareGPT5K)                                                                      | 90 K         | 多语言      | 实际用户-ChatGPT 对话                      | 是        |
|                                    | [WildChat](https://huggingface.co/datasets/allenai/WildChat)                                                                         | 150 K        | 多语言      | 实际用户-ChatGPT 对话                      | 是        |
|                                    | [WizardCoder (Luo et al., 2023)](https://github.com/lm-sys/FastChat/wiki/WizardCoder)                                               | –            | 代码        | LLaMA 2 生成                              | 否        |
|                                    | [Magicoder (Wei et al., 2023b)](https://github.com/ise-llc/magicoder?tab=readme#ov-file#dataset)                                    | 75 K / 110 K | 代码        | GPT-3.5 生成                              | 是        |
|                                    | [WaveCoder (Yu et al., 2023)](https://github.com/thinkabout-ai/wavecoder)                                                           | –            | 代码        | GPT-4 生成                                | 否        |
|                                    | [Phi-1 (Gunasekar et al., 2023)](https://huggingface.co/microsoft/phi-1)                                                             | 6 B Tokens   | 代码问答    | GPT-3.5 生成                              | 是        |
|                                    | [Phi-1.5 (Li et al., 2023i)](https://huggingface.co/microsoft/phi-1-5)                                                               | –            | 代码问答    | GPT-3.5 生成                              | 否        |
|                                    | [Nectar (Zhu et al., 2023a)](https://huggingface.co/datasets/berkeley-nest/Nectar)                                                   | 183 K        | 英文        | GPT-4 生成                                | 是        |
| **合成数据（自我增强）**             | [Self-Instruct (Wang et al., 2022c)](https://github.com/yizhongw/self-instruct)                                                      | 52 K         | 英文        | InstructGPT 生成                          | 是        |
|                                    | [Instruction Backtranslation (Li et al., 2023g)](https://github.com/tatsu-lab/stanford_alpaca)                                       | 502 K        | 英文        | LLaMA 生成                               | 否        |
|                                    | [SPIN (Chen et al., 2024b)](https://github.com/ucaml/SPIN?tab=readme#ov-file#Data)                                                    | 49.8 K       | 英文        | Zephyr 生成                              | 是        |


### 9.2.2 数据格式

为了便于加载和训练，通常将数据组织为 **JSONL**（每行一个 JSON 对象）或 **CSV**，关键字段包括：

| 字段           | 类型    | 说明                             |
| -------------- | ------- | -------------------------------- |
| `instruction`  | 字符串  | 用户层面的任务描述或问题             |
| `input`        | 字符串  | 可选的上下文信息（如文档片段、表格） |
| `output`       | 字符串  | 期望模型生成的回答或结果             |

#### JSONL 示例

```jsonl
{"instruction": "请用一句话概括以下文章 的核心观点。", "input": "人工智能正以前所未有的速度发展...", "output": "人工智能通过大规模预训练和多模态融合，推动了各行业的智能化转型。"}
{"instruction": "给定一个整数列表，返回其中的最大值。", "input": "[3, 7, 2, 9, 5]", "output": "9"}
```

### 9.2.3 数据质量与多样性
本节概述高质量指令微调数据集构建的六大关键要素，包括多领域覆盖、措辞多样、答案准确、避免偏见、样本平衡和持续迭代，为微调效果提供坚实保障。

1. **覆盖多领域、多任务**  
   高质量的微调数据集应覆盖写作辅助、问答检索、代码生成、对话管理和数据分析等多种任务，确保模型在不同场景下都有足够的样本进行学习。

2. **多样的提问措辞**  
   针对同一任务，使用“总结文章”“提炼要点”“写一段概要”等多种自然语言表述，让模型学会理解和处理不同的措辞变体。 

3. **高质量、准确的参考答案**  
   答案必须条理清晰、逻辑严谨且符合常识，代码示例需可执行并遵循最佳实践，对话回复需礼貌恰当，以保证模型学习到可靠的输出模式。 

4. **避免偏见与错误信息**  
   数据中不得包含歧视性、误导性或违法内容，尤其在医疗和法律等敏感领域，应谨慎处理或附加免责声明，以免模型产生有害偏见。 

5. **平衡数据分布**  
   通过对稀缺任务进行数据增强或控制常见任务比例，防止模型在训练中对某一类指令“偏科”，以获得更均衡的能力分布。 

6. **持续迭代与审校**  
   在初次微调后，应通过人工或自动化评估模型在各类指令上的表现，补充或优化效果不佳的样本，并定期清洗、去重和更新数据集。 

## 9.3 （实战）微调训练流程

在本节中，我们将演示如何利用 Hugging Face Transformers 的 Trainer 接口，在有限算力环境下完成一次小规模的指令微调实践。首先，我们会选择一个轻量级模型并设置合适的超参数，然后加载并预处理示例数据，调用 Trainer.train() 进行微调；接着介绍如何在训练中用验证集监控过拟合，并通过对比原始模型和微调模型的输出，直观感受到指令微调带来的改进；最后，我们给出在真实项目中提高效率和质量的实战建议，包括 LoRA、Adapter 以及持续迭代数据的思路。  


### 9.3.1 模型及参数设置

在开始微调前，需要根据你的硬件资源（主要是 GPU 显存）选择合适的模型，并设定关键超参数。下面先介绍如何选模型和测显存，再示范在代码中加载模型并配置 `TrainingArguments`。

#### 模型选择

| 模型           | 参数量      | 建议显存        | 说明                                                                                   |
| -------------- | ----------- | --------------- | -------------------------------------------------------------------------------------- |
| GPT-2 small    | 124M        | 6 – 28 GB       | Batch=1, SeqLen=1024 下实测约需 6.1 GB；Batch=8, SeqLen=1024 下实测约需 27.5 GB |
| DistilGPT2     | 82M         | 4 – 8 GB        | 通常 Batch=8, SeqLen=512 可在 8 GB 卡上运行                        |
| LLaMA-7B       | 7 B         | 14 – 28 GB      | 半精度浮点 (fp16) 约需 14 GB，单精度浮点 (fp32) 约需 28 GB |

*说明：显存需求受批次大小、序列长度、混合精度、激活检查点等多种因素影响，上表为常见配置的经验值；实际可根据自己环境做小批量测试。 如果你的 GPU 显存 ≤ 8 GB，优先选 GPT-2 或 DistilGPT2；如果有 16 GB 以上显存，可尝试 LLaMA-7B 的 fp16 版本。*

---

**模型和分词器加载（以GPT2为例）**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-large"  # 或者 "distilgpt2"、"wandb/lama-7b" 等

tokenizer = AutoTokenizer.from_pretrained(model_name)
model     = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer.pad_token_id = tokenizer.eos_token_id

### 关键超参数及影响

| 参数             | 建议范围       | 作用                                      | 影响                                          |
|--------------------------------|---------------|---------------------------------|---------------------------------------------|
| 学习率 (`learning_rate`)               | 1e-5 – 5e-5   | 控制每次梯度更新的步长                      | 学习率高时收敛更快但可能不稳定 :contentReference[oaicite:0]{index=0}        |
| 单卡批次大小 (`per_device_train_batch_size`) | 2 – 8         | 单个 GPU/TPU 核心或 CPU 上并行处理的样本数      | 增大可提升稳定性和吞吐，但显存/内存占用更高 :contentReference[oaicite:1]{index=1} |
| 训练轮数 (`num_train_epochs`)          | 1 – 3         | 数据完整过一遍的次数                        | 轮数过少可能欠拟合，过多易过拟合                  |
| 梯度累积 (`gradient_accumulation_steps`)| 1 – 8         | 在更新前累积多少个小批次的梯度，模拟更大批次      | 可在显存不足时提升“等效批次”，但训练速度变慢    :contentReference[oaicite:2]{index=2}      |
| 权重衰减 (`weight_decay`)              | 0 – 0.01      | L2 正则化项，用于防止过拟合                   | 小幅提升模型泛化能力            

---

**配置TrainingArguments**

In [ ]:
from transformers import TrainingArguments

# 超参数示例
learning_rate = 5e-5
batch_size    = 4
num_epochs    = 1
grad_accum    = 2
weight_decay  = 0.01

training_args = TrainingArguments(
    output_dir="./fine_tuned",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    logging_steps=10,
    save_steps=100,
    fp16=True,                  # 混合精度，加速并节省显存
    report_to="none"            # 关闭 WandB/其他报告，可按需启用
)

### 9.3.2 训练过程讲解
在指令微调阶段，我们不是像预训练那样简单地做下一个词预测（Next-Token Prediction），而是要让模型在完整的“指令+输入”上下文后，最大化人类标注的“回答”序列的对数似然。整体流程如下：  
1. **加载并预处理数据**：将 JSONL/CSV 格式的指令–回答对读入内存并构造 `Dataset`；  
2. **加载模型 & 配置超参**：选择合适的小模型（如 GPT-2）、定义 `TrainingArguments`；  
3. **损失函数 & 优化目标**：从数学上对比预训练和微调的交叉熵损失公式；  
4. **Trainer 训练演示**：使用 Hugging Face 的 `Trainer` 接口快速启动微调并观察日志。

---

**Step1. 加载并预处理数据**
在指令微调里，第一步是把提前整理好的“指令–输入–回答”对读入内存，转换成模型可接受的格式。我们使用 Hugging Face Datasets 库直接加载社区常用的 Stanford Alpaca 数据集，它包含约 52K 条由 GPT-3.5 生成并经人类审核的“指令–回答”对，格式包括 `instruction`、`input`（可为空）和 `output`。

In [7]:
from datasets import load_dataset

# 从 Hugging Face Hub 加载 Alpaca 指令微调数据集
alpaca = load_dataset("tatsu-lab/alpaca")

# 查看 train split 中首条样本的结构
sample = alpaca["train"][0]
print("指令：", sample["instruction"])
print("输入：", sample["input"])
print("参考回答：", sample["output"])

指令： Give three tips for staying healthy.
输入： 
参考回答： 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


In [8]:
def preprocess(examples):
    prompts = [
        f"### 指令：{ins}\n### 输入：{inp}\n### 回答：{out}"
        for ins, inp, out in zip(examples["instruction"], examples["input"], examples["output"])
    ]
    toks = tokenizer(prompts, truncation=True, padding="max_length", max_length=512)
    toks["labels"] = [
        [tok if tok != tokenizer.pad_token_id else -100 for tok in seq]
        for seq in toks["input_ids"]
    ]
    return toks

# 应用预处理并去除原字段
tokenized = alpaca.map(preprocess, batched=True, remove_columns=alpaca["train"].column_names)

# 预览样本
print(tokenized["train"][0])

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map: 100%|██████████| 52002/52002 [00:23<00:00, 2225.03 examples/s]

{'input_ids': [21017, 10545, 234, 229, 20015, 97, 171, 120, 248, 23318, 1115, 9040, 329, 10589, 5448, 13, 198, 21017, 5525, 122, 241, 17739, 98, 171, 120, 248, 198, 21017, 10263, 249, 252, 163, 18433, 171, 120, 248, 16, 13, 47659, 257, 12974, 5496, 290, 787, 1654, 284, 2291, 6088, 286, 15921, 290, 13701, 13, 220, 198, 17, 13, 32900, 7987, 284, 1394, 534, 1767, 4075, 290, 1913, 13, 220, 198, 18, 13, 3497, 1576, 3993, 290, 5529, 257, 6414, 3993, 7269, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 502

**Step2. 加载模型&配置超参数**
在9.3.1我们已成功加载好模型并利用TrainingArguments配置了训练超参数。

**Step3：使用 Trainer 启动微调**
将模型、超参和数据集传给 Trainer，一行 .train() 启动完整训练循环，并在控制台打印 loss 变化。

In [9]:
from transformers import Trainer

# 实例化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=None   # 如需验证，可参考 9.3.3
)

# 启动微调
train_result = trainer.train()
print("最终训练损失：", train_result.training_loss)

[2025-07-04 09:30:24,692] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.6
 [WARNING]  using untested triton version (3.2.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/usr/local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][bootstrap.cc:56] NCCL INFO Bootstrap : Using eth0:22.29.241.210<0>
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][net.cc:229] NCCL INFO NET/Plugin: Failed to find ncclCollNetPlugin_v6 symbol.
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][net.cc:234] NCCL INFO NET/Plugin: Failed to find ncclCollNetPlugin symbol (v4 or v5).
dsw-282424-c4d84f94d-9tg4h accl-lite version: v1.1.157.g20f49ee
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][pfInit:1113] NCCL INFO NET/accl-lite unicm bind ipv4 22.29.241.210
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][pfInit:1383] NCCL INFO NET/accl-lite Init 1 SOE devices:
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][pfInit:1384] NCCL INFO NET/accl-lite +---+----------+
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][pfInit:1389] NCCL INFO NET/accl-lite | 0 | vsolar_0 |
[dsw-282424-c4d84f94d-9tg4h][2175613:2175613][0][pfInit:1390] NCCL INFO NET/accl-lite +---+----------+
[dsw-282424-c4d84f94d-9tg4h

...

[dsw-282424-c4d84f94d-9tg4h][2175613:2493482][0][init.cc:1882] NCCL INFO ncclCommInitRank comm 0x55fcf5067ce0 rank 0 nranks 4 hggcDev 0 hgmlDev 0 busId 10 commHash 917bbb0142a143d5 - Init START
[dsw-282424-c4d84f94d-9tg4h][2175613:2493483][1][init.cc:1882] NCCL INFO ncclCommInitRank comm 0x55fcf506dcb0 rank 1 nranks 4 hggcDev 1 hgmlDev 1 busId 20 commHash 917bbb0142a143d5 - Init START
[dsw-282424-c4d84f94d-9tg4h][2175613:2493484][2][init.cc:1882] NCCL INFO ncclCommInitRank comm 0x55fcf5074a00 rank 2 nranks 4 hggcDev 2 hgmlDev 2 busId 30 commHash 917bbb0142a143d5 - Init START
[dsw-282424-c4d84f94d-9tg4h][2175613:2493485][3][init.cc:1882] NCCL INFO ncclCommInitRank comm 0x55fcf507b750 rank 3 nranks 4 hggcDev 3 hgmlDev 3 busId 40 commHash 917bbb0142a143d5 - Init START
[dsw-282424-c4d84f94d-9tg4h][2175613:2493484][2][init.cc:137] NCCL INFO gTaskid = 917bbb0142a143d5.
[dsw-282424-c4d84f94d-9tg4h][2175613:2493482][0][init.cc:1266] NCCL INFO icnScaleOutSupport : NO

[dsw-282424-c4d84f94d-9tg4

/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,1.815300
20,1.487000
30,1.532700
40,1.500400
50,1.517900
60,1.438800
70,1.454800
80,1.440300
90,1.440800
100,1.457000


/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

最终训练损失： 1.359107208251953


## 9.4 微调效果对比与分析 
下面我们用几个最直观的小测试——“摘要”、“改写”、“翻译”、“问答”和“情感判别”这五种常见场景，来看看微调前后的模型差别。每个测试都很简单，直接给指令就能跑，大家可以一目了然地感受到模型对指令的理解力和输出质量究竟有没有提升。

In [ ]:
import transformers
from transformers import pipeline

# 1. 加载基线模型 GPT2-large
generator_baseline = pipeline(
    "text-generation",
    model="gpt2-large",
    tokenizer="gpt2",
    device=0  # 如果没有 GPU，请改为 device=-1
)

# 2. 加载微调后模型
generator_ft = pipeline(
    "text-generation",
    model="./fine_tuned/checkpoint-1600",
    tokenizer="gpt2-large",
    device=0
)

# 3. 准备测试示例
tests = {
    "Summarization": (
        "Instruction: Summarize the following article in no more than two sentences.\n"
        "Input: The rapid advancement of artificial intelligence has led to breakthroughs in natural language processing, computer vision, and robotics, enabling new applications across healthcare, finance, transportation, and education. These developments are improving efficiency, reducing human error, and opening up opportunities for personalized services and innovative research.\n"
    ),
    "Paraphrase": (
        "Instruction: Paraphrase the following sentence without changing its meaning:\n"
        "Input: The quick brown fox jumps over the lazy dog."
    ),
    "Translation": (
        "Instruction: Translate the following English sentence into Chinese:\n"
        "Input: I love exploring new technologies and sharing knowledge with others."
    ),
    "Factual QA": (
        "Instruction: What is the capital city of Canada?"
    ),
    "Sentiment Analysis": (
        "Instruction: Determine whether the sentiment of the following review is Positive or Negative:\n"
        "Input: The new smartphone model exceeded all my expectations with its performance."
    ),
}

# 4. 逐个生成并对比结果
for name, prompt in tests.items():
    print(f"=== {name} ===")
    print(f"Prompt: {prompt}\n")
    
    # 基线输出
    out_base = generator_baseline(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print("GPT2-Large 基线输出：")
    print(out_base, "\n")
    
    # 微调后输出
    out_ft = generator_ft(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]
    print("Finetuned GPT2-Large 输出：")
    print(out_ft, "\n")
    
    print("-" * 40 + "\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


=== Summarization ===
Prompt: Instruction: Summarize the following article in no more than two sentences.
Input: The rapid advancement of artificial intelligence has led to breakthroughs in natural language processing, computer vision, and robotics, enabling new applications across healthcare, finance, transportation, and education. These developments are improving efficiency, reducing human error, and opening up opportunities for personalized services and innovative research.




Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT2-Large 基线输出：
Instruction: Summarize the following article in no more than two sentences.
Input: The rapid advancement of artificial intelligence has led to breakthroughs in natural language processing, computer vision, and robotics, enabling new applications across healthcare, finance, transportation, and education. These developments are improving efficiency, reducing human error, and opening up opportunities for personalized services and innovative research.
Output: A study of nearly a thousand patients with dementia (those who have died) found that the ability to recognize faces 



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

GPT2-Large 基线输出：
Instruction: What is the capital city of Canada?

Dennis Allen: Nova Scotia

If this is your first time visiting, can you go by road?

My road is closed due to snow. I'm on Halifax Street.

If you need a ride to downtown Halifax, you're on the same road as I am: Main Street.

You can look to your left for a map. There is a bike share station at Main/Horseshoe 



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Finetuned GPT2-Large 输出：
Instruction: What is the capital city of Canada?
### 指令：This list is comprised of four countries: Canada, the United States of America, Mexico, and Japan.
### 输入：
### 回答：The capital city of Canada is Ottawa. The other four countries located in this list are Canada (Ottawa), the United States of America (New York City), Mexico (Mexico City), and 

----------------------------------------

=== Sentiment Analysis ===
Prompt: Instruction: Determine whether the sentiment of the following review is Positive or Negative:
Input: The new smartphone model exceeded all my expectations with its performance.



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT2-Large 基线输出：
Instruction: Determine whether the sentiment of the following review is Positive or Negative:
Input: The new smartphone model exceeded all my expectations with its performance.
Output: An "A Great" mood, because the new device is a lot of fun to use. But it's not perfect, it's just a "great" type of device.
A "Great" mood is in a positive state. Negative is in a depressed state.
Conclusion: The device is a lot of fun 

Finetuned GPT2-Large 输出：
Instruction: Determine whether the sentiment of the following review is Positive or Negative:
Input: The new smartphone model exceeded all my expectations with its performance.
### 指令：
### 输入：The new smartphone model exceeded all my expectations with its performance.
### 回答：Negative sentiment. The new smartphone has unsatisfactory processing speed, poor reliability, and can't handle any prolonged use. Other reviews 

----------------------------------------



在这几个简单的例子里，我们能明显感觉到微调前后的差别：

- 在“摘要”任务里，未经微调的 GPT-2 往往东拉西扯，提到不相干的内容；而微调后的模型虽然还是会中途截断，但它至少抓住了“AI 带来 NLP、视觉和机器人突破”这一核心点。
- “改写”这条简单的句子，基线模型只是机械地换词，语义走样；微调后它会尝试多种同义表达，虽然有时还是不够精准，但整体感觉更贴近原句。
- 做“翻译”时，基线版本总爱加一堆前缀“Result: …”，完全不像翻译；微调后它直接进入中文输出，格式整洁多了，虽然翻得还不够地道。
- “事实问答”环节，基线模型甚至跑去编故事，微调后它能说出“Ottawa”，但有时会再多扯几句别的国家，显得不够简洁。
- “情感分类”里，基线喜欢用长段文字抒发情绪，微调后它能给出“Negative”标签，但紧跟着的解释往往莫名其妙，说明它还没完全理解“分类”这件事。

总体来说，指令微调后的模型在“听懂指令”上确实有了进步，但还不是特别稳：它能找到关键点，却常常苦于句子连贯和多余信息的问题。这些差异背后有几方面原因：
- 首先，GPT-2 Large（774M 参数）在面对 Alpaca 这种涵盖摘要、翻译、分类等多样化指令的 52K 样本时，容量仍显不足，尤其在深度理解和长文本生成上容易捉襟见肘。
- 其次，Alpaca 数据由 text-davinci-003 自动生成，示例质量参差，有时含格式错误或不准确信息，模型在全量微调中会“学坏”这些噪声。
- 再者，一次性全参数微调虽能贴合训练集，却易引发过拟合，使得模型丧失预训练时的通用语言能力，在部分场景下输出跑题或重复输入。
- 另一方面，若超参数（学习率、batch size、epoch 数）未做精细化调优，训练过程会出现不稳定或收敛不充分的问题。
- 最后，在推理阶段若不结合 beam search、多样性/重复惩罚及格式正则化，生成时仍会出现诸如“Result: …”前缀、过长或跑题的现象。

对此，我们可以尝试一些针对上述问题的改进方案：
- 参数高效微调：使用 LoRA、Prefix-Tuning 或 Adapter，仅微调少部分参数即可快速适配指令任务，兼顾预训练知识保留与过拟合风险控制。
- 数据清洗与扩增：剔除格式异常、逻辑混乱的示例，并结合 Self-Instruct 等自我增强方法自动生成并筛选高质量数据，以提升训练信号纯度。
- 分阶段训练：先用小学习率和少量 epoch 做适配器微调，再在此基础上进行短轮次全量微调或 RLHF 强化，分层保留通用能力并增强指令对齐性。
- 精细化超参数调优：采用学习率衰减（如 cosine annealing）、梯度累积和 checkpointing，结合 Early Stopping，避免训练过拟合或不足。
- 生成控制与后处理：在推理时启用 beam search、多样性惩罚和重复惩罚，并做简单的格式正则化过滤，去除多余前缀和跑题内容。
- 容量升级与持续评估：如条件允许，可升级至 GPT-2 XL 或更大模型缓解容量瓶颈，并结合 ROUGE、BLEU 等自动指标及人工评测，持续监控与优化输出质量。

## 9.5 实践建议与拓展

1. **增量微调与模型复用**  
在真实项目中，指令微调通常不是从零开始训练，而是基于已有的微调模型（如 LLaMA-Adapter、Stanford Alpaca 等）进行二次增量训练。你可以先在社区或企业开源的指令微调 checkpoint 上，用自己的业务数据做少量训练，既保留了成熟模型的通用能力，又显著节省了算力和时间。此外，建议先在 GPT-2、DistilGPT2 等小参数量模型上验证数据管道和超参数，再将成熟流程迁移到大模型，实现“小模型打样，大模型生产”的高效迭代。

2. **数据管道与自动化构建**  
构建大规模、高质量的指令–回答对，需要借助混合标注和自动化流水线。常见做法是先用预训练或初步微调模型自动生成大量“指令–回答”对，再通过关键词过滤、重复度检测或小型分类器初筛，最后由人工审核确保准确性和多样性。在生产环境中，可使用 Apache Airflow、Prefect 等轻量级 ETL 工具，搭建“提取—清洗—格式化—质检—入库”流水线，并定期抽样检查数据健康度，一旦发现问题立即触发补采或人工复审。

3. **轻量化微调技术**  
当算力受限或需调教更大模型时，LoRA（Low-Rank Adaptation）、Adapter 和 PEFT 等技术提供了高效方案。它们只训练少量新增权重，却能达到接近全量微调的效果，将可训练参数减少千倍以上、显存需求降低三倍以上。结合混合精度训练（fp16/bf16）和梯度累积，你甚至可以在单卡 12 GB 显存的环境下，顺利微调数十亿参数的模型。

4. **部署与迭代闭环**  
在模型上线后，应结合线上 A/B 测试和人工评估收集真实用户反馈，并将失败案例快速纳入训练集，形成“训练—部署—监控—补充”的持续迭代闭环。此外，可使用 INT8/GPTQ 等量化方法，以及 ONNX Runtime、TensorRT 等推理引擎，将微调模型压缩并部署到边缘设备或多卡集群，实现更灵活、高效的生产化落地。

## 9.6 小结
在本章中，我们从概念、数据、流程、效果对比到实践建议，完整地演示了如何在预训练模型之上，使用有监督的“指令–回答”对对模型进行微调。首先，我们介绍了指令微调的核心目标——提高模型对指令意图的可控性、一致性和对人类偏好的贴合度；接着详细说明了从 Hugging Face、Alpaca、Dolly 等渠道获取数据、组织成JSONL格式并保证多领域、多措辞、高质量与公平性的要点；然后，通过 Trainer 接口示例，指导读者选择适合算力的小模型，配置学习率、批次大小、训练轮数等超参数，完成分词、标签对齐、训练与验证，展示了如何监控 loss 曲线并防止过拟合；最后，我们通过原始模型与微调模型的对比示例，直观感受到了指令微调在输出准确度和条理性上的提升，并给出 LoRA、Adapter、自动化数据生成和持续迭代等实践技巧。通过本章学习，你已经掌握了动手构建指令微调管道的全流程，为更高效的 LoRA 微调与模型量化（第十章）打下了坚实基础。

## 9.7 课后习题

以下习题分为三大部分：选择题、简答题和编程题，共9题，用于巩固书籍第9章“指令微调”内容。

### 一、选择题

1. **单项选择**：下列哪项**不是**指令微调相比预训练的优势？
   A. 更好控制生成输出
   B. 降低模型参数总量
   C. 增强对指令一致性理解
   D. 提高符合人类偏好的程度

2. **单项选择**：在微调阶段，为什么要将 `pad_token_id` 替换为 -100？
   A. 节省显存
   B. 让 CrossEntropyLoss 忽略填充位置
   C. 加速推理
   D. 改善模型收敛速度

3. **多项选择**：关于生成指令微调数据集的方法，下列说法正确的有：
   A. 从头人工编写示例
   B. 利用大模型作为教师蒸馏生成
   C. 让模型在自身生成的数据上迭代完善
   D. 直接使用原始无标注语料即可

4. **单项选择**：以下哪种操作能让模型在微调时仅计算回答部分的 loss？
   A. 将 `pad_token_id` 替换为 -100
   B. 调整学习率曲线
   C. 增加训练数据量
   D. 启用梯度累积

5. **单项选择**：下列关于 JSONL 格式的描述，哪项**正确**？
   A. 每行一个 JSON 对象
   B. 必须使用 JSON 数组包裹所有对象
   C. 不支持中文字符
   D. 文件中不得包含换行符



### 二、简答题

1. 简述指令微调与预训练的核心区别，并说明为什么只对回答部分计算损失能提升模型表现。

2. 结合本章内容，设计3条指令–回答示例，涵盖不同任务（如摘要、翻译、代码生成），并简要说明每条示例的目的和预期效果。



### 三、编程题

1. 编写 Python 函数，将一个包含 `instruction`、`input`、`output` 字段的 `list[dict]` 保存为 `examples.jsonl` 文件，并保证中文正常显示，每行一个 JSON 对象。

2. 基于本章 `preprocess` 示例，完成以下函数：

   ```python
   def preprocess(examples):
       # 拼接 instruction/input/output 为提示文本
       # 使用 tokenizer 编码并截断/填充
       # 生成 labels，将 pad_token_id 替换为 -100
       # 返回包含 input_ids、attention_mask、labels 的字典
   ```



> **提示**：编程题可在 Jupyter Notebook 中验证，简答题可书面阐述。


## 9.7 参考文献
1. Wei, J., Bosma, M., Zhao, V. Y., Guu, K., Yu, A. W., Lester, B., Du, N., Dai, A. M., & Le, Q. V. (2021). _Finetuned Language Models Are Zero-Shot Learners_. arXiv preprint arXiv:2109.01652. [https://arxiv.org/abs/2109.01652]  
2. Ouyang, L., Wu, J., Jiang, X., Almeida, D., Wainwright, C. L., Mishkin, P., Zhang, C., Agarwal, S., Slama, K., Ray, A., Schulman, J., Hilton, J., Kelton, F., Miller, L., Simens, M., Askell, A., Welinder, P., Christiano, P., Leike, J., & Lowe, R. (2022). _Training language models to follow instructions with human feedback_. NeurIPS 2022; arXiv:2203.02155. [https://arxiv.org/abs/2203.02155]  
3. Chung, H. W., Hou, L., Longpre, S., Zoph, B., Tay, Y., Fedus, W., Li, Y., Wang, X., Dehghani, M., Brahma, S., Webson, A., Gu, S. S., Dai, Z., Suzgun, M., Chen, X., Chowdhery, A., Castro-Ros, A., Pellat, M., Robinson, K., … Wei, J. (2022). _Scaling Instruction-Finetuned Language Models_. arXiv preprint arXiv:2210.11416. [https://arxiv.org/abs/2210.11416]  
4. Peng, B., Li, C., He, P., Galley, M., & Gao, J. (2023). _Instruction Tuning with GPT-4_. arXiv preprint arXiv:2304.03277. [https://arxiv.org/abs/2304.03277]  
5. Wang, Y., Kordi, Y., Mishra, S., Liu, A., Smith, N. A., Khashabi, D., & Hajishirzi, H. (2022). _Self-Instruct: Aligning Language Models with Self-Generated Instructions_. arXiv preprint arXiv:2212.10560. [https://arxiv.org/abs/2212.10560]  
6. Zhang, S., Dong, L., Li, X., Zhang, S., Sun, X., Wang, S., Li, J., Hu, R., Zhang, T., Wu, F., & Wang, G. (2023). _Instruction Tuning for Large Language Models: A Survey_. arXiv preprint arXiv:2308.10792. [https://arxiv.org/abs/2308.10792] :contentReference[oaicite:0]{index=0}  
